In [31]:
#Imports

import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import numpy as np
import itertools
from sklearn.metrics import classification_report
import statsmodels.api as sm

# jesse. jesse we have to train. 

In [32]:
full_drugs = pd.read_csv(r'full_drug_dataset.csv')

In [33]:
cols = []
featured = full_drugs.iloc[:, full_drugs.columns != 'Amphet']
for column in featured:
    cols.append(column)
label = 'Amphet'

full_drugs['Amphet'] = full_drugs['Amphet'].map({'CL0': 0, 
                                            'CL1': 0, 
                                            'CL2': 1, 
                                            'CL3': 1, 
                                            'CL4': 1, 
                                            'CL5': 1,
                                            'CL6': 1})
X = full_drugs[cols]
y = full_drugs[label]


In [34]:
logit_model=sm.Logit(y,X)               # runs the model
result=logit_model.fit()
results_summary = result.summary2()

#print(p_values_tailored)


Optimization terminated successfully.
         Current function value: 0.534957
         Iterations 6


In [35]:
p_values_tables = results_summary.tables[1]['P>|z|'].tolist()

p_values = {}                   # dictionary of the p_values
for i in range(0, len(p_values_tables)):
    p_values[results_summary.tables[1]['P>|z|'].index[i]] = p_values_tables[i]
    #p_values[i]# p_values_tables[i]

p_values_tailored = {}          # dictionary of all p_values under 0.05, ie the ones that are statistically significant
for i in range(0, len(p_values_tables)):
    if(list(p_values.values())[i] < 0.05):
        p_values_tailored[results_summary.tables[1]['P>|z|'].index[i]] = list(p_values.values())[i]